In [615]:
import utils
import pandas as pd
import numpy as np

alerts, jams = utils.load_data()

In [616]:
alerts.shape

(8567, 16)

In [617]:
alerts.head()

,country,city,reportRating,reportByMunicipalityUser,confidence,reliability,type,uuid,roadType,magvar,subtype,street,location,pubMillis,endreport,reportDescription
0,CI,Antofagasta,1,false,5,10,HAZARD,a123f22e-e5e0-4c6c-8a4e-7434c4fd2110,6,17,HAZARD_ON_ROAD_CONSTRUCTION,Av. República de Croacia,"{'x': -70.412429, 'y': -23.676846}",1724417637000,1724425125000,NaN
1,CI,Antofagasta,0,false,0,5,JAM,1d8df954-e380-4327-8015-f4fbadf74f2d,6,4,JAM_HEAVY_TRAFFIC,Av. Edmundo Pérez Zujovic,"{'x': -70.392283, 'y': -23.617475}",1724423878000,1724425125000,NaN
2,CI,Antofagasta,0,false,0,5,JAM,c53a5bc6-f6b6-44b8-a5bd-af69b60a4580,1,359,,Pasaje El Volcán,"{'x': -70.385619, 'y': -23.573323}",1724425406000,1724425125000,NaN
3,CI,Antofagasta,1,false,0,5,JAM,53b6ff5f-d078-421d-9f78-03e534d78989,2,1,,Azapa,"{'x': -70.384986, 'y': -23.616328}",1724425622000,1724425125000,NaN
4,CI,Antofagasta,2,false,0,5,ACCIDENT,e87552f7-4572-4096-9fdd-9400ad90e7f8,2,281,,Irarrázaval,"{'x': -70.385121, 'y': -23.603942}",1724425939000,1724425125000,NaN


In [618]:
alerts_cleaned = utils.extract_event(alerts, ["JAM", "ACCIDENT"], extra_col=['type']).drop('uuid', axis=1)
alerts_cleaned['day'] = alerts_cleaned.inicio.dt.day
alerts_cleaned['month'] = alerts_cleaned.inicio.dt.month
alerts_cleaned['year'] = alerts_cleaned.inicio.dt.year
alerts_cleaned['minute'] = alerts_cleaned.inicio.dt.minute
alerts_cleaned = alerts_cleaned.drop(['inicio', 'fin', 'geometry'], axis=1)

alerts_cleaned.head()

,street,x,y,type,hour,week_day,day_type,day,month,year,minute
1,Av. Edmundo Pérez Zujovic,-70.392283,-23.617475,JAM,10,4,s,23,8,2024,37
2,Pasaje El Volcán,-70.385619,-23.573323,JAM,11,4,s,23,8,2024,3
3,Azapa,-70.384986,-23.616328,JAM,11,4,s,23,8,2024,7
4,Irarrázaval,-70.385121,-23.603942,ACCIDENT,11,4,s,23,8,2024,12
5,Coquimbo,-70.399240,-23.657366,JAM,11,4,s,23,8,2024,40


In [619]:
alerts_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6136 entries, 1 to 8563
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   street    6111 non-null   object 
 1   x         6136 non-null   float64
 2   y         6136 non-null   float64
 3   type      6136 non-null   object 
 4   hour      6136 non-null   int32  
 5   week_day  6136 non-null   int32  
 6   day_type  6136 non-null   object 
 7   day       6136 non-null   int32  
 8   month     6136 non-null   int32  
 9   year      6136 non-null   int32  
 10  minute    6136 non-null   int32  
dtypes: float64(2), int32(6), object(3)
memory usage: 431.4+ KB


In [620]:
print(f"{len(alerts_cleaned[alerts_cleaned['street'].isna()]) / len(alerts_cleaned['street']) * 100:.2f}% de lo datos es nulos en 'street'")

0.41% de lo datos es nulos en 'street'


In [621]:
alerts_cleaned["street"].value_counts()[:10]

street
Av. Edmundo Pérez Zujovic    1537
Av. Pedro Aguirre Cerda       548
Av. Grecia                    402
Av. Iquique                   304
Av. Antonio Rendic            275
Av. Balmaceda                 169
Nicolás Tirado                154
Av. Argentina                 124
Av. Óscar Bonilla             120
Av. Salvador Allende           83
Name: count, dtype: int64

In [622]:
alerts_cleaned["street"].value_counts().index[:10]

Index(['Av. Edmundo Pérez Zujovic', 'Av. Pedro Aguirre Cerda', 'Av. Grecia',
       'Av. Iquique', 'Av. Antonio Rendic', 'Av. Balmaceda', 'Nicolás Tirado',
       'Av. Argentina', 'Av. Óscar Bonilla', 'Av. Salvador Allende'],
      dtype='object', name='street')

In [623]:
# Eliminamos valores nulos y filtro por calle

streets = alerts_cleaned["street"].value_counts().index[:10].to_numpy()

alerts_cleaned = alerts_cleaned[alerts_cleaned['street'].apply(lambda x: x in streets)]
alerts_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3716 entries, 1 to 8551
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   street    3716 non-null   object 
 1   x         3716 non-null   float64
 2   y         3716 non-null   float64
 3   type      3716 non-null   object 
 4   hour      3716 non-null   int32  
 5   week_day  3716 non-null   int32  
 6   day_type  3716 non-null   object 
 7   day       3716 non-null   int32  
 8   month     3716 non-null   int32  
 9   year      3716 non-null   int32  
 10  minute    3716 non-null   int32  
dtypes: float64(2), int32(6), object(3)
memory usage: 261.3+ KB


In [624]:
alerts_cleaned.to_csv('alerts.csv', sep=',', decimal='.', encoding='utf-8', index=False)

In [625]:
cat = ["street", "type", "day_type"]
num = list(alerts_cleaned.drop(cat, axis=1).columns)

In [626]:
# Balanceo de eventos y creación de no eventos

events = alerts_cleaned.copy()
events['happen'] = 1

q_events = len(events)

street = events['street']
x = events['x']
y = events['y']
type = np.random.choice(["ACCIDENT", "JAM"], q_events)
hour = np.random.randint(events.hour.min(), events.hour.max(), q_events)
minute = np.random.randint(events.minute.min(), events.minute.max(), q_events)
week_day = np.random.randint(events.week_day.min(), events.week_day.max(), q_events)
day_type = np.random.choice(["s", "f"], q_events)
day = np.random.randint(events.day.min(), events.day.max() + 1, q_events)
month = np.random.randint(events.month.min(), events.month.max() + 1, q_events)
year = np.random.choice(events.year.unique(), q_events)


no_events = pd.DataFrame({
    "street": street,
    "x": x,
    "y": y,
    "type": type,
    "hour": hour,
    "minute": minute,
    "week_day": week_day,
    "day_type": day_type,
    "day": day,
    "month": month,
    "year": year,
    "happen": 0
})

no_events

,street,x,y,type,hour,minute,week_day,day_type,day,month,year,happen
1,Av. Edmundo Pérez Zujovic,-70.392283,-23.617475,JAM,14,17,5,s,21,10,2024,0
7,Av. Grecia,-70.402438,-23.650212,JAM,8,7,1,f,25,8,2024,0
8,Av. Grecia,-70.402429,-23.649991,ACCIDENT,16,31,0,f,29,9,2024,0
9,Av. Balmaceda,-70.400096,-23.645631,ACCIDENT,20,18,2,s,28,10,2024,0
11,Av. Balmaceda,-70.401486,-23.647052,ACCIDENT,16,20,2,s,5,9,2024,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8543,Av. Pedro Aguirre Cerda,-70.388947,-23.616659,ACCIDENT,4,33,4,s,5,10,2024,0
8544,Av. Grecia,-70.402335,-23.647936,ACCIDENT,22,33,0,f,6,8,2024,0
8545,Av. Pedro Aguirre Cerda,-70.391009,-23.576996,JAM,22,32,3,f,27,10,2024,0
8548,Av. Edmundo Pérez Zujovic,-70.395761,-23.637618,ACCIDENT,15,0,4,s,7,9,2024,0


In [627]:
total_events = pd.concat([events, no_events], axis=0)
total_events['happen'].value_counts()

happen
1    3716
0    3716
Name: count, dtype: int64

In [628]:
# Se elimina mes y año porque no hay muestras suficientes

X_happen = total_events.drop(['happen', "type", "x", "y", "street", "month", "year", "day"], axis=1)
y_happen = total_events['happen']

In [629]:
X_happen["day_type"] = X_happen["day_type"].map({"f": 0, "s": 1})


In [630]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train_happen, X_test_happen, y_train_happen, y_test_happen = train_test_split(X_happen, y_happen, test_size=0.2, random_state=42)

rfc_happen = RandomForestClassifier(random_state=42)
rfc_happen.fit(X_train_happen, y_train_happen)

y_predict_happen = rfc_happen.predict(X_test_happen)

In [631]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score

confusion_matrix(y_test_happen, y_predict_happen)

array([[585, 147],
       [125, 630]])

In [632]:
recall = recall_score(y_test_happen, y_predict_happen)
precision = precision_score(y_test_happen, y_predict_happen)
f1 = f1_score(y_test_happen, y_predict_happen)

recall, precision, f1

(np.float64(0.8344370860927153),
 np.float64(0.8108108108108109),
 np.float64(0.8224543080939948))

In [633]:
# Un dato de prueba

test = {
    "hour": [7],
    "week_day": [4],
    "day_type": [1],
    "minute": [20]
}

rfc_happen.predict_proba(pd.DataFrame(test)), rfc_happen.predict(pd.DataFrame(test))


(array([[0.16, 0.84]]), array([1]))

In [634]:
X_type = events.drop(["happen", "day", "month", "year", "x", "y", "type"], axis=1)
y_type = events["type"]

In [635]:
X_type["day_type"] = X_type["day_type"].map({"f": 0, "s": 1})

In [636]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X_type['street'] = le.fit_transform(X_type["street"])
y_type = le.fit_transform(y_type)

In [637]:
X_train_type, X_test_type, y_train_type, y_test_type = train_test_split(X_type, y_type, test_size=0.2, random_state=42)

In [638]:
rfc_type = RandomForestClassifier(random_state=42)
rfc_type.fit(X_train_type, y_train_type)

y_predict_type = rfc_type.predict(X_test_type)

In [641]:
confusion_matrix(y_test_type, y_predict_type)

array([[ 16,  60],
       [ 23, 645]])

In [642]:
recall = recall_score(y_test_type, y_predict_type)
precision = precision_score(y_test_type, y_predict_type)
f1 = f1_score(y_test_type, y_predict_type)

recall, precision, f1

(np.float64(0.9655688622754491),
 np.float64(0.9148936170212766),
 np.float64(0.9395484340859432))